# Prompt Templates and Prompt Values

In [3]:
from langchain import PromptTemplate

In [4]:
TEMPLATE = '''
System:
{description}

Human:
I've recently adopted a {pet}.
Could you suggest some {pet} names?
'''


In [5]:
prompt_template = PromptTemplate.from_template(TEMPLATE)

In [6]:
prompt_template

PromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, template="\nSystem:\n{description}\n\nHuman:\nI've recently adopted a {pet}.\nCould you suggest some {pet} names?\n")

In [ ]:
# PromptTemplate のインスタンスに対して、引数を指定して呼び出すことで、PromptValue を生成できます。
prompt_value = prompt_template.invoke({'description':'You are a helpful assistant.',
                                      'pet':'dog'})

In [11]:
# TEMPLATE の変数に 渡した dict の内容が反映されている
prompt_value

StringPromptValue(text="\nSystem:\nYou are a helpful assistant.\n\nHuman:\nI've recently adopted a dog.\nCould you suggest some dog names?\n")

In [13]:
%load_ext dotenv
%dotenv

import os
from openai import AzureOpenAI
from azure.identity import ClientSecretCredential, get_bearer_token_provider
# Azure OpenAI Service Integration with LangChain
from langchain_openai.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=6
)

In [18]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
TEMPLATE_S = '{description}'
TEMPLATE_H = '''I've recently adopted a {pet}. Could you suggest some {pet} names?'''

In [19]:
message_template_s = SystemMessagePromptTemplate.from_template(TEMPLATE_S)
message_template_h = HumanMessagePromptTemplate.from_template(TEMPLATE_H)


In [27]:
type(message_template_h)

langchain_core.prompts.chat.HumanMessagePromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [message_template_s, message_template_h]
)
# ChatPromptTemplate 型で返ってくる
type(chat_template)

langchain_core.prompts.chat.ChatPromptTemplate

In [22]:
chat_value = chat_template.invoke({
    "description": "You are a helpful assistant.",
    "pet": "dog"
})

In [23]:
chat_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a dog. Could you suggest some dog names?", additional_kwargs={}, response_metadata={})])

In [ ]:
# ChatPromptValue を LLM に渡すと、AIMessage が返ってきます。
ai_msg = llm.invoke(chat_value)

In [26]:
ai_msg

AIMessage(content="Congratulations on adopting a dog! Here are some name suggestions to help you find the perfect fit for your new furry friend. I've categorized them to make it easier for you:\n\n### **Classic Dog Names**\n- Max\n- Bella\n- Charlie\n- Daisy\n- Buddy\n- Lucy\n- Rocky\n- Molly\n\n### **Unique Dog Names**\n- Nova\n- Atlas\n- Juniper\n- Zephyr\n- Indigo\n- Echo\n- Aspen\n- Onyx\n\n### **Cute Dog Names**\n- Peanut\n- Coco\n- Teddy\n- Poppy\n- Biscuit\n- Nala\n- Mochi\n- Sprout\n\n### **Strong Dog Names**\n- Thor\n- Athena\n- Diesel\n- Roxy\n- Titan\n- Zara\n- Bear\n- Juno\n\n### **Funny Dog Names**\n- Bark Twain\n- Sir Waggington\n- Chewbarka\n- Furdinand\n- Paws\n- Waffles\n- Nugget\n- Pickles\n\n### **Nature-Inspired Names**\n- Willow\n- River\n- Maple\n- Clover\n- Storm\n- Sunny\n- Sage\n- Luna\n\n### **Pop Culture-Inspired Names**\n- Arya (Game of Thrones)\n- Yoda (Star Wars)\n- Simba (The Lion King)\n- Elsa (Frozen)\n- Scooby (Scooby-Doo)\n- Loki (Marvel)\n- Stitch (L

# Few Shot Chat Message Prompt Templates

In [31]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    FewShotChatMessagePromptTemplate
)

In [32]:
TEMPLATE_H = '''I've recently adopted a {pet}. Could you suggest some {pet} names?'''
TEMPLATE_AI = '''{response}'''

message_template_h = HumanMessagePromptTemplate.from_template(TEMPLATE_H)
message_template_a = AIMessagePromptTemplate.from_template(TEMPLATE_AI)

In [35]:
example_template = ChatPromptTemplate.from_messages([message_template_h, message_template_a])

In [46]:
examples = [
    {"pet": "dog", "response": "Here are some dog names: Max, Bella, Charlie."},
    {"pet": "cat", "response": "Here are some cat names: Whiskers, Luna, Oliver."}
]

In [47]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt = example_template
)

In [48]:
few_shot_prompt

FewShotChatMessagePromptTemplate(examples=[{'pet': 'dog', 'response': 'Here are some dog names: Max, Bella, Charlie.'}, {'pet': 'cat', 'response': 'Here are some cat names: Whiskers, Luna, Oliver.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['pet', 'response'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}. Could you suggest some {pet} names?"), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['response'], input_types={}, partial_variables={}, template='{response}'), additional_kwargs={})]))

In [49]:
# few_shot_template と message_template_h を組み合わせて、最終的なチャットテンプレートを作成します。
chat_template = ChatPromptTemplate.from_messages([few_shot_prompt, message_template_h])
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[FewShotChatMessagePromptTemplate(examples=[{'pet': 'dog', 'response': 'Here are some dog names: Max, Bella, Charlie.'}, {'pet': 'cat', 'response': 'Here are some cat names: Whiskers, Luna, Oliver.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['pet', 'response'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}. Could you suggest some {pet} names?"), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['response'], input_types={}, partial_variables={}, template='{response}'), additional_kwargs={})])), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template="I've recently adopted a {pet}

In [50]:
# message_template_h には、まだ引数が渡されていないため、invoke()
chat_value = chat_template.invoke({'pet': 'rabbit'})
chat_value


ChatPromptValue(messages=[HumanMessage(content="I've recently adopted a dog. Could you suggest some dog names?", additional_kwargs={}, response_metadata={}), AIMessage(content='Here are some dog names: Max, Bella, Charlie.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a cat. Could you suggest some cat names?", additional_kwargs={}, response_metadata={}), AIMessage(content='Here are some cat names: Whiskers, Luna, Oliver.', additional_kwargs={}, response_metadata={}), HumanMessage(content="I've recently adopted a rabbit. Could you suggest some rabbit names?", additional_kwargs={}, response_metadata={})])

In [51]:
for i in chat_value.messages:
    print(f'{i.type}: {i.content}\n')

human: I've recently adopted a dog. Could you suggest some dog names?

ai: Here are some dog names: Max, Bella, Charlie.

human: I've recently adopted a cat. Could you suggest some cat names?

ai: Here are some cat names: Whiskers, Luna, Oliver.

human: I've recently adopted a rabbit. Could you suggest some rabbit names?



In [52]:
ai_msg = llm.invoke(chat_value)

In [54]:
ai_msg

AIMessage(content="Congratulations on adopting a rabbit! Here are some fun and adorable rabbit name suggestions:\n\n### For Male Rabbits:\n- Thumper  \n- Hopper  \n- Peter (inspired by Peter Rabbit)  \n- Bun Bun  \n- Oreo  \n- Jasper  \n- Basil  \n- Clover  \n- Dusty  \n- Finn  \n\n### For Female Rabbits:\n- Daisy  \n- Hazel  \n- Buttercup  \n- Willow  \n- Mochi  \n- Poppy  \n- Snowflake  \n- Rosie  \n- Maple  \n- Nibbles  \n\n### Gender-Neutral Names:\n- Flopsy  \n- Marshmallow  \n- Cinnabun  \n- Peanut  \n- Cotton  \n- Twilight  \n- Fudge  \n- Patches  \n- Cloud  \n- Sprout  \n\nFeel free to mix and match or get creative based on your rabbit's personality and appearance! 🐇", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 170, 'prompt_tokens': 88, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_to